# Screen EFMs for cost and yield

In [1]:
import cobra
import pandas as pd
import numpy as np
import sys
sys.path.append('../../utils')
import EC_utils
import importlib
importlib.reload(EC_utils)
import multiprocessing as mp

## Load EC Model

In [2]:
ec_model=cobra.io.read_sbml_model('../../Model/EC_iCH360/EC_iCH360_fitted_kapps.xml')

#Read fitted scaling factors for aerobic growth on glucose
fitted_scalings=pd.read_csv('../../Enzyme_Constraints/kcat_adjustment_and_enzyme_allocation_predictions/out/condition_specific_scalings.csv',index_col=0)

#Parse reaction costs from the model
ec_cost=EC_utils.get_reaction_costs(ec_model,sigma=float(fitted_scalings.loc['glc__D_e']))


Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-12


## Load EFMs

In [3]:
carbon_source='EX_glc__D_e'
carbon_condition='glc__D_aerobic'
efms=pd.concat([pd.read_csv('../../EFM/efm_enumeration/efms/ich360red_glc__D_aerobic_efms.csv',index_col=0),
                pd.read_csv('../../EFM/efm_enumeration/efms/ich360red_glc__D_anaerobic_efms.csv',index_col=0)
                ],
                )
chunks=np.array_split(efms,mp.cpu_count())
del efms

## Compute yield and cost of each mode with a parallel Pool

In [4]:
pool=mp.Pool(mp.cpu_count())

In [5]:
f=EC_utils.compute_efm_df_yield_cost
flux_supporting=['EX_ac_e','EX_o2_e','EX_glc__D_e','ATPM','Biomass']
args=[(chunk,ec_cost,'Biomass',carbon_source,flux_supporting) for chunk in chunks]

parallel_computation=pool.starmap(f,args)
out=list(parallel_computation)

In [6]:
yield_cost_df=pd.concat(out)
yield_cost_df.to_csv(f'out/ich360red_{carbon_condition}_yield_cost.csv')
yield_cost_df.head()

,yield,cost,EX_ac_e,EX_o2_e,EX_glc__D_e,ATPM,Biomass
0,0.042102,10.040055,0.580546,-24.733394,-23.751971,0.0,1.0
1,0.040818,12.662366,0.580546,-31.18622,-24.498839,0.0,1.0
2,0.034503,14.920963,0.0,-36.77204,-28.982787,0.0,1.0
3,0.042102,29.830455,0.580546,-24.733394,-23.751971,0.0,1.0
4,0.053609,13.716434,0.580546,-34.034239,-18.653427,0.0,1.0


In [7]:
del yield_cost_df

### Repeat computation under low Oxygen conditions

In [8]:
ec_cost_low_o2=ec_cost.copy()
for cytochrome_rxn in ['CYTBO3_4pp_fw','CYTBDpp_fw','CYTBD2pp_fw']:
    ec_cost_low_o2[cytochrome_rxn]=1000*ec_cost[cytochrome_rxn]

pool=mp.Pool(mp.cpu_count())
f=EC_utils.compute_efm_df_yield_cost
flux_supporting=['EX_ac_e','EX_o2_e','EX_glc__D_e','ATPM','Biomass']
args=[(chunk,ec_cost_low_o2,'Biomass',carbon_source,flux_supporting) for chunk in chunks]

parallel_computation=pool.starmap(f,args)
out_low_o2=list(parallel_computation)

In [9]:
yield_cost_df_low_o2=pd.concat(out_low_o2)
yield_cost_df_low_o2.to_csv(f'out/ich360red_{carbon_condition}_yield_cost_low_o2.csv')

In [10]:
del yield_cost_df_low_o2